In [ ]:
import numpy as np
import pandas as pd
from datetime import datetime
import os
import matplotlib.pyplot as plt
import matplotlib as mpl
import openfilegui


In [ ]:
file = openfilegui.gui_fname("").decode()


In [ ]:
df = pd.read_csv(file)
df['Start'] = pd.to_datetime(df['Start'])
df['End'] = pd.to_datetime(df['End'])
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]  # удалаяем битые столбцы
df['Total'] = ((df['End']-df['Start']).dt.total_seconds() / 60).astype(int)

activities = df.Activity.unique()

first_year = df.Start.min().year
last_year = df.End.max().year

cdf = pd.read_csv('data/colors.txt')
color_dict = dict(cdf.values)

In [ ]:
def get_by_week(year):
    weeks = range(1, 54)
    dfy = pd.DataFrame(0, index=np.arange(len(weeks)),
                    columns=np.hstack(['week', activities]))
    dfy['week'] = weeks
    dfy = dfy.set_index('week')

    for index, row in df.iterrows():
        if (row.Start.year > year):
            continue
        if (row.Start.year < year):
            break

        dfy[row.Activity].iloc[row.Start.week - 1] += row.Total

    dfy = dfy.loc[:, (dfy != 0).any(axis=0)]  # удаляем столбцы, в которых все нули
    dfy.iloc[:, :] = dfy.iloc[:, :].div(60, axis=0) # minutes to hours

    colors = []

    for c in dfy.columns:
        if c in color_dict:
            colors.append(color_dict[c])
        else:
            # colors.append("orange")
            dfy.drop(columns=[c], inplace=True)
            # print(f"Dropped {c} column!")

    # os.makedirs('data', exist_ok=True)
    # dfy.to_csv('data/' + str(year) + '.csv', index=False)

    return dfy, colors

In [ ]:
def get_by_month(year):
    months = range(1, 13)
    dfy = pd.DataFrame(0, index=np.arange(len(months)),
                       columns=np.hstack(['month', activities]))
    dfy['month'] = months
    dfy = dfy.set_index('month')

    for index, row in df.iterrows():
        if (row.Start.year > year):
            continue
        if (row.Start.year < year):
            break
        dfy[row.Activity].iloc[row.Start.month - 1] += row.Total

    # удаляем столбцы, в которых все нули
    dfy = dfy.loc[:, (dfy != 0).any(axis=0)]
    dfy.iloc[:, :] = dfy.iloc[:, :].div(60, axis=0)  # minutes to hours

    colors = []

    for c in dfy.columns:
        if c in color_dict:
            colors.append(color_dict[c])
        else:
            dfy.drop(columns=[c], inplace=True)

    return dfy, colors

In [ ]:
def get_by_month_for_all():
    years = []
    months = []

    for year in range(first_year, last_year + 1):
        for month in range(1, 13):
            years.append(year)
            months.append(month)

    data = {'year': years, 'month': months}

    dfy = pd.DataFrame(data, columns=np.hstack(['year', 'month', activities]))
    dfy.fillna(0, inplace=True)

    for index, row in df.iterrows():
        # dfy.loc[:, ((dfy['year'] == row.Start.year) & (
        #     dfy['month'] == row.Start.month - 1), row.Activity)] += row.Total
        i = (row.Start.year - first_year) * 12 + row.Start.month - 1
        dfy.at[i, row.Activity] += row.Total

    # dfy['ni'] = dfy['year'] + '-' + dfy['month']
    # dfy = dfy.assign(ni=lambda x: x.year + '-' + x.month)
    dfy['year'] = dfy['year'].astype('str')
    dfy['month'] = dfy['month'].astype('str')
    dfy['ni'] = dfy[['year', 'month']].agg('-'.join, axis=1)
    dfy = dfy.set_index('ni')

    # удаляем столбцы, в которых все нули
    dfy = dfy.loc[:, (dfy != 0).any(axis=0)]

    colors = []
    for c in dfy.columns:
        if c in color_dict:
            colors.append(color_dict[c])
        else:
            dfy.drop(columns=[c], inplace=True)

    # удаляем строки, в которых все нули
    dfy = dfy.loc[~(dfy == 0).all(axis=1)]

    # minutes to hours
    dfy.iloc[:, :] = dfy.iloc[:, :].div(60, axis=0)


    return dfy, colors


In [ ]:
for year in range(first_year, last_year + 1):
    dfy, colors = get_by_week(year)
    dfy.plot(kind='area', stacked=False, figsize=(
        20, 10), color=colors, linewidth=0)
    plt.title(year)
    plt.ylabel('hours')
    plt.xlabel('week')
    plt.show()

In [ ]:
for year in range(first_year, last_year + 1):
    dfy, colors = get_by_month(year)
    dfy.plot(kind='area', stacked=False, figsize=(
        20, 10), color=colors, linewidth=0)
    plt.title(year)
    plt.ylabel('hours')
    plt.xlabel('month')
    plt.show()

In [ ]:
dfy, colors = get_by_month_for_all()
dfy.plot(kind='area', stacked=False, figsize=(
    20, 10), color=colors, linewidth=0)
plt.title('all')
plt.ylabel('hours')
plt.xlabel('month')
plt.show()

In [ ]:
dfy, colors = get_by_month_for_all()
dfy.plot(kind='area', stacked=True, figsize=(
    20, 10), color=colors, linewidth=0)
plt.title('all')
plt.ylabel('hours')
plt.xlabel('month')
plt.show()